In [1]:
import OSLPP

In [2]:
domains = ['art', 'clipart', 'product', 'real_world']
for source in domains:
    for target in domains:
        if not source == target:
            params = OSLPP.Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, 
                  dataset='OfficeHome', source=source, target=target,
                  num_src_classes=25, num_total_classes=65)
            print(source, target, OSLPP.main(params))

art clipart OS=54.53 OS*=54.00 unk=67.92 HOS=60.16
art product OS=72.75 OS*=72.67 unk=74.76 HOS=73.70
art real_world OS=79.66 OS*=80.01 unk=70.70 HOS=75.07
clipart art OS=50.04 OS*=48.89 unk=79.00 HOS=60.40
clipart product OS=63.20 OS*=62.70 unk=75.47 HOS=68.50
clipart real_world OS=68.00 OS*=67.73 unk=74.55 HOS=70.98
product art OS=55.02 OS*=54.13 unk=77.43 HOS=63.71
product clipart OS=51.39 OS*=50.71 unk=68.40 HOS=58.24
product real_world OS=77.32 OS*=77.46 unk=73.68 HOS=75.53
real_world art OS=61.88 OS*=61.35 unk=75.26 HOS=67.60
real_world clipart OS=55.36 OS*=54.99 unk=64.65 HOS=59.43
real_world product OS=77.74 OS*=78.03 unk=70.42 HOS=74.03
